### Oil type by origin
Goal: Determine oil type weighting in tank traffic by destination and vessel type


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [2]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

In [3]:
# Import columns are: (E) 'StartDateTime, (G) Deliverer, (H) Receiver, (O) Region, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
# Use region to isolate terminals in Salish Sea only (by excluding non-Salish-Sea transfer sites)
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="E,G,H,O,P,Q,R,W,X")

### create a list of all terminals included in the dataset, together with Region 
(so we can isolate Salish Sea) 

In [4]:
# The following list includes facilities used in Casey's origin/destination analysis 
# with names matching the Dept. of Ecology (DOE) database.  
# For example, the shapefile "Maxum Petroleum - Harbor Island Terminal" is labeled as 'Maxum (Rainer Petroleum)'
# in the DOE database.  I use the Ecology language here and will need to translate to Shapefile speak 
facility_names = [ 'BP Cherry Point Refinery', 'Shell Puget Sound Refinery', 'Shell Oil LP Seattle Distribution Terminal',
                   'Maxum (Rainer Petroleum)', 'Tidewater Snake River Terminal','Nustar Energy Tacoma', 
                   'SeaPort Sound Terminal', 'Tesoro Vancouver Terminal','Phillips 66 Ferndale Refinery', 
                   'Phillips 66 Tacoma Terminal', 'Andeavor Anacortes Refinery (formerly Tesoro)',
                   'Tesoro Port Angeles Terminal','U.S. Oil & Refining','US Oil Tacoma ','Naval Air Station Whidbey Island (NASWI)',
                   'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 'Kinder Morgan Liquids Terminal - Harbor Island',
                   'Nustar Energy Tacoma', 'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 'Tidewater Vancouver Terminal',
                   'TLP','TLP Management Services LLC (TMS)']

## Create dictionary for fuel information

In [5]:
[nrows,ncols] = df.shape

# Create a destination dictionary
destination = {}
destination['names'] = facility_names #####  REMOVE TLP AND US OIL TACOMA FROM THIS LIST  ####

# Create an origin dictionary
origin = {}
origin['names'] = facility_names  #####  REMOVE TLP AND US OIL TACOMA FROM THIS LIST  ####

# define vessel and fuel types to include in .yaml file
vessel_types = ['atb', 'tug', 'tanker']
fuel_type    = ['crude', 'bunker', 'jet', 'diesel', 'gas', 'other']
data_type    = ['total_gallons', 'percent_of_total', 'number_of_transfers']

for rows in range(nrows):
  
    # Create dictionary structure for destination
    if df.Receiver[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if df.Receiver[rows] == 'TLP' or df.Receiver[rows] == 'TLP Management Services LLC (TMS)':
            label = 'TLP Management Services LLC (TMS)'
        elif df.Receiver[rows] == 'U.S. Oil & Refining' or df.Receiver[rows] == 'US Oil Tacoma':
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[rows]}'
        
        # create dictionaries for each location     
        destination[f'{label}'] = {}

        for vessel in vessel_types:
            destination[f'{label}'][f'{vessel}'] = {}
            for fuel in fuel_type:
                destination[f'{label}'][f'{vessel}'][f'{fuel}'] = {}
                for data in data_type:
                    destination[f'{label}'][f'{vessel}'][f'{fuel}'][f'{data}'] = 0

 # Create dictionary structure for origin
    if df.Deliverer[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if df.Deliverer[rows] == 'TLP' or df.Deliverer[rows] == 'TLP Management Services LLC (TMS)':
            label = 'TLP Management Services LLC (TMS)'
        elif df.Deliverer[rows] == 'U.S. Oil & Refining' or df.Deliverer[rows] == 'US Oil Tacoma':
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[rows]}'
        
        # create dictionaries for each location     
        origin[f'{label}'] = {}
        
        for vessel in vessel_types:
            origin[f'{label}'][f'{vessel}'] = {}
            for fuel in fuel_type:
                origin[f'{label}'][f'{vessel}'][f'{fuel}'] = {}
                for data in data_type:
                    origin[f'{label}'][f'{vessel}'][f'{fuel}'][f'{data}'] = 0


## Loop through data again and catalog fuel information

In [6]:
## Loop through data again and catalog fuel information

####  NOTE: I USE ROWS ABOVE AND ROW HERE.  THAT'S CRAZY MAKING.  I PLAN TO FIX. 
for row in range(nrows):
    
    # Catalogue fuel imports (destination traffic)
    if df.Receiver[row] in facility_names and df.TransferType[row] == 'Cargo':
        
        if df.Receiver[row] == 'TLP' or df.Receiver[row] == 'TLP Management Services LLC (TMS)':
            label = 'TLP Management Services LLC (TMS)'
        elif df.Receiver[row] == 'U.S. Oil & Refining' or df.Receiver[row] == 'US Oil Tacoma':
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.DelivererTypeDescription[row] == 'TANK SHIP':

            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[f'{label}']['tanker']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['crude']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[f'{label}']['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[f'{label}']['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[f'{label}']['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[f'{label}']['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['diesel']['number_of_transfers'] += 1

            else:
                destination[f'{label}']['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tanker']['other']['number_of_transfers'] += 1

        # ATBs
        elif df.DelivererTypeDescription[row] == 'TANK BARGE' and 'ATB' in df.Deliverer[row]:
            
             # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[f'{label}']['atb']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['atb']['crude']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[f'{label}']['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[f'{label}']['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[f'{label}']['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['atb']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[f'{label}']['tug']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['diesel']['number_of_transfers'] += 1

            else:
                destination[f'{label}']['tug']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif df.DelivererTypeDescription[row] == 'TANK BARGE' and 'ATB' not in df.Deliverer[row]:
            
            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[f'{label}']['tug']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['crude']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[f'{label}']['tug']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[f'{label}']['tug']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[f'{label}']['tug']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[f'{label}']['tug']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['diesel']['number_of_transfers'] += 1

            else:
                destination[f'{label}']['tug']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                destination[f'{label}']['tug']['other']['number_of_transfers'] += 1
            
        else:
            print('non-tank delivery...interesting....')
            
   # Catalogue fuel exports (origin traffic)
    if df.Deliverer[row] in facility_names and df.TransferType[row] == 'Cargo':
        
        if df.Deliverer[row] == 'TLP' or df.Deliverer[row] == 'TLP Management Services LLC (TMS)':
            label = 'TLP Management Services LLC (TMS)'
        elif df.Deliverer[row] == 'U.S. Oil & Refining' or df.Deliverer[row] == 'US Oil Tacoma':
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.ReceiverTypeDescription[row] == 'TANK SHIP':

            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[f'{label}']['tanker']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['crude']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[f'{label}']['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[f'{label}']['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[f'{label}']['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[f'{label}']['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['diesel']['number_of_transfers'] += 1

            else:
                origin[f'{label}']['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tanker']['other']['number_of_transfers'] += 1
        
        # ATBs
        elif df.ReceiverTypeDescription[row] == 'TANK BARGE' and 'ATB' in df.Receiver[row]:
            
             # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[f'{label}']['atb']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['atb']['crude']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[f'{label}']['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[f'{label}']['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[f'{label}']['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['atb']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[f'{label}']['tug']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['diesel']['number_of_transfers'] += 1

            else:
                origin[f'{label}']['tug']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif df.ReceiverTypeDescription[row] == 'TANK BARGE' and 'ATB' not in df.Receiver[row]:
            
            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[f'{label}']['tug']['crude']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['crude']['number_of_transfers'] += 1
                print(origin[f'{label}']['tug']['crude']['total_gallons'])
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[f'{label}']['tug']['bunker']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[f'{label}']['tug']['gas']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[f'{label}']['tug']['jet']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[f'{label}']['tug']['diesel']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['diesel']['number_of_transfers'] += 1

            else:
                origin[f'{label}']['tug']['other']['total_gallons'] += df.TransferQtyInGallon[row]
                origin[f'{label}']['tug']['other']['number_of_transfers'] += 1
            
        #else:
          #  print('non-tank transfer from facility...interesting....')

non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
3444000
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
non-tank delivery...interesting....
6804000
non-tank del

In [15]:
origin['Phillips 66 Ferndale Refinery']['atb']['crude']

{'total_gallons': 0, 'percent_of_total': 0, 'number_of_transfers': 0}

In [19]:
destination['Phillips 66 Ferndale Refinery']['tanker']['crude']['total_gallons']

560490000

In [18]:
with open(r'./cargo_by_origin.yaml', 'w') as file:
    documents = yaml.dump(origin, file)
with open(r'./cargo_by_destination.yaml', 'w') as file:
    documents = yaml.dump(destination, file)